<a href="https://colab.research.google.com/github/najla933/ArabicRNN/blob/master/Najla_SemanticSimilarity_Arabic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

the paper : https://www.aaai.org/ocs/index.php/AAAI/AAAI16/paper/view/12195  
 
 https://github.com/arpanmukherjee/Detecting-Semantic-Similarity-in-Questions#lstm-siamese-network 

In [ ]:
pip install tensorflow==1.15.4
pip install tensorboard==1.12.0
pip install pandas==0.23.4
pip install Keras==2.2.4
pip install numpy
pip install nltk==3.2 
pip install gensim==3.6.0

In [ ]:
from google.colab import drive
drive.mount('/content/drive/',force_remount=True)

Mounted at /content/drive/


In [ ]:
import requests
url = 'https://archive.org/download/AraVec/Twt-CBOW.zip'
r = requests.get(url, allow_redirects=True)

In [ ]:
open('/content/drive/MyDrive/Twt-CBOW.zip', 'wb').write(r.content)

369123021

In [ ]:
# !wget -P /root/input/ -c https://archive.org/download/AraVec/Twt-CBOW.zip
!ls /content/drive/MyDrive/Twt-CBOW.zip
!unzip /content/drive/MyDrive/Twt-CBOW.zip -d /content/drive/MyDrive/Twt-CBOW.zip

In [ ]:
from time import time
import pandas as pd
import numpy as np
from gensim.models import KeyedVectors
from keras.preprocessing.sequence import pad_sequences
from keras.models import Model
from keras.layers import Input, Embedding, LSTM, Lambda
import keras.backend as K
from keras.optimizers import Adadelta
from keras.callbacks import ModelCheckpoint
import tensorflow as tf
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sns
import os, itertools, datetime, re, string, sys, argparse, nltk
from gensim.models import Word2Vec  
from sklearn.model_selection import train_test_split

In [ ]:
datasetFile = open('/content/drive/MyDrive/Dataset-DL001.csv')
X = pd.read_csv(datasetFile)

split= 0.8
splitAt = int(0.8*len(X))
train_df = X[0:splitAt]
test_df = X[splitAt:]

print(train_df.columns)

EMBEDDING_FILE = '/content/drive/MyDrive/Colab Notebooks/SemanticSimilarity/Twt-CBOW'
MODEL_SAVING_DIR = 'drive/MyDrive/Colab Notebooks/'

Index(['Question', 'Answer', 'Label'], dtype='object')


In [ ]:
def text_to_word_list(text):
    text = str(text)

    text = re.sub(r"ى", "ي", text)
    text = re.sub(r"ؤ", "ء", text)
    text = re.sub(r"ئ", "ء", text)
    text = re.sub(r"ة", "ه", text)
    text = re.sub(r"گ", "ك", text)
    text = re.sub(r",", " ", text)
    text = re.sub(r"\.", " ", text)
    text = re.sub(r"!", " ! ", text)
    text = re.sub(r"؟", " ", text)
    text = re.sub(r"\/", " ", text)
    text = re.sub(r"\^", " ^ ", text)
    text = re.sub(r"\+", " + ", text)
    text = re.sub(r"\-", " - ", text)
    text = re.sub(r"\=", " = ", text)
    text = re.sub(r"/:", " ", text)
    text = re.sub(r"'", " ", text)
    text = re.sub(r":", " : ", text)
    text = text.split()
    return text

In [ ]:
nltk.download('stopwords')
stops = set(stopwords.words('arabic'))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
vocabulary = dict()
inverse_vocabulary = ['<unk>'] 
word2vec = Word2Vec.load(EMBEDDING_FILE)
print(len(word2vec.wv.vocab))

164077


In [ ]:
questions_cols =['Question','Answer']

for dataset in [train_df, test_df]:
  for index, row in dataset.iterrows(): 
    for question in questions_cols :
      # Iterate through the text of both questions of the row
      q2n = []  # q2n -> question numbers representation
      for word in text_to_word_list(row[question]): 
          if word in stops and word not in word2vec.wv.vocab: # Check for unwanted words 
              continue
          if word not in vocabulary:
              vocabulary[word] = len(inverse_vocabulary)
              q2n.append(len(inverse_vocabulary))
              inverse_vocabulary.append(word)
          else:
              q2n.append(vocabulary[word])

      print(f"Index:{index}, Question: {row[question]}, Q2N: {q2n}" )
      dataset.at[index,question]=q2n 

embedding_dim = 300  #using 300 dimension for my embedding i.e. there will 300 vectors for each word in the corpora represented for neural network model.
embeddings = 1 * np.random.randn(len(vocabulary)+ 1, embedding_dim) #embedding matrix ###len(vocabulary)
embeddings[0] = 0 #padding will be ignored 

#Build the embedding matrix
for word, index in vocabulary.items():
    if word in word2vec.wv.vocab:
        embeddings[index] = word2vec.wv.word_vec(word)
# del word2vec

Index:0, Question: ما هو حق المؤلف؟, Q2N: [1, 2, 3, 4]
Index:0, Question: يهدف حق المؤلف إلى حماية المؤلفين (الكتاب، الفنانون، الملحنون) في ابتكاراتهم. ويطلق على هذه
الابتكارات عادة اسم "المصنفات"., Q2N: [5, 3, 4, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]
Index:1, Question: ما هو حق المؤلف؟, Q2N: [1, 2, 3, 4]
Index:1, Question: أتاحت القوانين الدولية وعدد من القوانين واللوائح الإقليمية والوطنية ودون الوطنية، وضع إطار التطبيق مبدأ الموافقة المسبقة والمستنيرة للنفاذ إلى الموارد الوراثية، وتحديد شروط هذا النفاذ، لا سيما ضمان استخدام هذه الموارد استخدامة مستدامة واقتسام منافعه بإنصاف. وتشمل العناصر الرئيسية من القانون الدولي، اتفاقية التنوع البيولوجي والمعاهدة الدولية بشأن الموارد الوراثية النباتية للأغذية والزراعة التابعة لمنظمة الأغذية والزراعة. وفي إطار الاتفاقات المبرمة بشأن النفاذ إلى الموارد الوراثية واقتسام المنافع والنظم القانونية التي تتيح هذا النفاذ، تكتسي التدابير الخاصة المتصلة بإدارة الملكية الفكرية أهمية بالغة لأنها تضمن مراعاة هذه الاتفاقات لضرورة أن يعود النفا

In [ ]:
max_seq_length = max(train_df['Question'].map(lambda x: len(x)).max(),
                     train_df['Answer'].map(lambda x: len(x)).max(),
                      test_df['Question'].map(lambda x: len(x)).max(),
                      test_df['Answer'].map(lambda x: len(x)).max())

In [ ]:
trainX = train_df[questions_cols]
trainY = train_df['Label']
testX = test_df[questions_cols]
testY = test_df['Label']

In [ ]:
# Split to dicts
X_train = {'left': trainX.Question, 'right': trainX.Answer}
X_test = {'left': testX.Question, 'right': testX.Answer}

Y_train = trainY.values
Y_test = testY.values  
 # Zero padding
for dataset, side in itertools.product([X_train, X_test], ['left', 'right']):
    dataset[side] = pad_sequences(dataset[side], maxlen=max_seq_length)

In [ ]:
assert X_train['left'].shape == X_train['right'].shape
assert len(X_train['left']) == len(Y_train)

In [ ]:
n_hidden = 50
gradient_clipping_norm = 1.25

#Keras doesn't come with Manhattan distance calculation, hence we need to write a routine to do that for us.
def exponent_neg_manhattan_distance(left, right):
    return K.exp(-K.sum(K.abs(left-right), axis=1, keepdims=True))

#Let's build the model now:
#The visible layer
left_input = Input(shape=(max_seq_length,), dtype='int32')
right_input = Input(shape=(max_seq_length,), dtype='int32')

embedding_layer = Embedding(len(embeddings), embedding_dim, weights=[embeddings], input_length=max_seq_length, trainable=False)

# Embedded version of the inputs
encoded_left = embedding_layer(left_input)
encoded_right = embedding_layer(right_input)

# Since this is a siamese network, both sides share the same LSTM
shared_lstm = LSTM(n_hidden)

left_output = shared_lstm(encoded_left)
right_output = shared_lstm(encoded_right)

# Calculates the distance as defined by the MaLSTM model
malstm_distance = Lambda(function=lambda x: exponent_neg_manhattan_distance(x[0], x[1]),output_shape=lambda x: (x[0][0], 1))([left_output, right_output])


# Pack it all up into a model
malstm = Model([left_input, right_input], [malstm_distance]) 


In [ ]:
malstm.compile(
    optimizer=Adadelta(clipnorm=gradient_clipping_norm, ),
    loss='mean_squared_error', metrics=['accuracy']
)
print(malstm.summary())

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 138)]        0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, 138)]        0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, 138, 300)     715200      input_1[0][0]                    
                                                                 input_2[0][0]                    
__________________________________________________________________________________________________
lstm (LSTM)                     (None, 50)           70200       embedding[0][0]              

In [ ]:
malstm_trained = malstm.fit([X_train['left'], X_train['right']], Y_train, batch_size=64, 
                            validation_data=([X_test['left'], X_test['right']], Y_test), epochs=4)

Epoch 1/4
24/24 [==============================] - 10s 315ms/step - loss: 0.2580 - accuracy: 0.7420 - val_loss: 0.2480 - val_accuracy: 0.7500
Epoch 2/4
24/24 [==============================] - 7s 274ms/step - loss: 0.2455 - accuracy: 0.7544 - val_loss: 0.2480 - val_accuracy: 0.7500
Epoch 3/4
24/24 [==============================] - 6s 271ms/step - loss: 0.2461 - accuracy: 0.7539 - val_loss: 0.2480 - val_accuracy: 0.7500
Epoch 4/4
24/24 [==============================] - 7s 276ms/step - loss: 0.2604 - accuracy: 0.7396 - val_loss: 0.2480 - val_accuracy: 0.7500
